In [17]:
import pandas as pd
import numpy as np

In [8]:
data=pd.read_csv('/content/sample_data/Movie_Compiled.csv')
data.head()

,Rating,Date,Title,Review
0,,12 April 2012,An Unstoppable Force of Awesomeness! All Hail ...,I have got to tell anyone who will listen that...
1,,19 April 2012,They balanced everything masterfully. As Good ...,"If you're a fan of epic adventure movies, then..."
2,10.0,20 April 2012,The Avengers assembled flawlessly,I'm sorry to say The Avengers isn't a good mov...
3,9.0,16 April 2012,LA Review of 'The Avengers' (No spoilers),LA Review of 'The Avengers' (No spoilers)\r\n\...
4,10.0,2 July 2017,Bad-ass descent-amazing superhero MCU action f...,Marvel's The Avengers (2012) is an awesome des...


In [10]:
#This is a sparse matrix
#This is rating prediction for those that are missing
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.metrics import cohen_kappa_score

In [11]:
data_1 = data[~data['Review'].isnull()]
data_1.shape

(4402, 4)

In [12]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4402 entries, 0 to 4501
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rating  4075 non-null   object
 1   Date    4402 non-null   object
 2   Title   4402 non-null   object
 3   Review  4402 non-null   object
dtypes: object(4)
memory usage: 172.0+ KB


In [13]:
data_1.Rating.value_counts()

10.0    1143
1.0      719
9.0      428
8.0      301
7.0      288
6.0      252
5.0      232
3.0      225
2.0      218
4.0      202
          67
Name: Rating, dtype: int64

In [26]:
data_2=data_1.replace(' ',np.nan)
data_2.head()

,Rating,Date,Title,Review
0,NaN,12 April 2012,An Unstoppable Force of Awesomeness! All Hail ...,I have got to tell anyone who will listen that...
1,NaN,19 April 2012,They balanced everything masterfully. As Good ...,"If you're a fan of epic adventure movies, then..."
2,10.0,20 April 2012,The Avengers assembled flawlessly,I'm sorry to say The Avengers isn't a good mov...
3,9.0,16 April 2012,LA Review of 'The Avengers' (No spoilers),LA Review of 'The Avengers' (No spoilers)\r\n\...
4,10.0,2 July 2017,Bad-ass descent-amazing superhero MCU action f...,Marvel's The Avengers (2012) is an awesome des...


In [27]:
data_2["Rating"]=data_2["Rating"].astype(float)
data_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4402 entries, 0 to 4501
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Rating  4008 non-null   float64
 1   Date    4402 non-null   object 
 2   Title   4402 non-null   object 
 3   Review  4402 non-null   object 
dtypes: float64(1), object(3)
memory usage: 172.0+ KB


In [35]:
train_data=data_2[~data_2['Rating'].isnull()]
test_data=data_2[data_2['Rating'].isnull()]

In [36]:
(train_data.shape,test_data.shape)

((4008, 4), (394, 4))

In [37]:
train_data

,Rating,Date,Title,Review
2,10.0,20 April 2012,The Avengers assembled flawlessly,I'm sorry to say The Avengers isn't a good mov...
3,9.0,16 April 2012,LA Review of 'The Avengers' (No spoilers),LA Review of 'The Avengers' (No spoilers)\r\n\...
4,10.0,2 July 2017,Bad-ass descent-amazing superhero MCU action f...,Marvel's The Avengers (2012) is an awesome des...
6,10.0,15 April 2012,A Comic Book Triumph!,I was lucky enough to be included in the group...
8,9.0,14 April 2012,Nothing Short of Amazing,I just saw the early screening for San Diego t...
...,...,...,...,...
4496,10.0,17 August 2005,A little story,After an attack by a pack of saber-toothed cat...
4498,9.0,25 June 2004,Great characters in a fun adventure!,"""Ice Age"" gives us a sabertooth tiger, a sloth..."
4499,10.0,1 June 2004,"""We are definitely the weirdest tribe I've eve...","Ice Age was a very entertaining movie, with br..."
4500,9.0,25 April 2004,This was a cool movie,"Ice Age, thats a good title. I was very confus..."


In [33]:
x_train, x_val, y_train, y_val = train_test_split(train_data.iloc[:,1:], train_data.iloc[:,0], test_size = 0.4, random_state= 123456)

In [39]:
from wordcloud import STOPWORDS

In [40]:
tfidf= TfidfVectorizer(stop_words=STOPWORDS,token_pattern="[a-z']+",max_df=0.55,min_df=10)

In [41]:
tfidf_mat_train=tfidf.fit_transform(x_train['Review'])
tfidf_mat_val=tfidf.transform(x_val['Review'])
tfidf_mat_test=tfidf.transform(test_data['Review'])

In [42]:
tfidf_mat_train.shape

(2404, 3657)

In [46]:
lr_model_1=LogisticRegression()
lr_model_1.fit(X=tfidf_mat_train,y=y_train)

LogisticRegression()

In [47]:
pred_val_lr=lr_model_1.predict(tfidf_mat_val)
cohen_kappa_score(pred_val_lr,y_val)

0.2431569431210462

In [50]:
N=150
svd=TruncatedSVD(n_components=N)
svd_mat_train=svd.fit_transform(tfidf_mat_train)
svd_mat_val=svd.transform(tfidf_mat_val)
svd_mat_test=svd.transform(tfidf_mat_test)

In [51]:
lr_model_2=LogisticRegression()
lr_model_2.fit(X=svd_mat_train,y=y_train)
pred_val_lr2=lr_model_2.predict(svd_mat_val)
cohen_kappa_score(pred_val_lr2,y_val)

0.2492717895322457

In [52]:
gbm=GradientBoostingClassifier(learning_rate=0.01,n_estimators=2000,max_depth=1)

In [53]:
gbm_1=gbm.fit(X=svd_mat_train,y=y_train)
pred_val_gbm1=gbm_1.predict(svd_mat_val)
cohen_kappa_score(pred_val_gbm1,y_val)

0.24227467461597363

In [54]:
svd_new=TruncatedSVD(n_components=150)

In [55]:
word_vec=svd_new.fit_transform(tfidf_mat_train.T)

In [56]:
word_vec.shape

(3657, 150)

In [58]:
word_vec

array([[ 0.16663287, -0.02542932, -0.02796946, ..., -0.00240115,
         0.03390854,  0.00924004],
       [ 0.07443251, -0.05981512, -0.06848025, ..., -0.04360545,
        -0.04948453, -0.00129531],
       [ 0.14860485,  0.00560963,  0.03498259, ...,  0.04322354,
         0.02165124,  0.01626362],
       ...,
       [ 0.12356809, -0.06661446, -0.07678164, ...,  0.00933361,
        -0.00595019, -0.00576774],
       [ 0.05873907, -0.03514347, -0.04228365, ...,  0.00525081,
         0.01147605, -0.00033372],
       [ 0.02800249, -0.01459836, -0.00817321, ..., -0.00214096,
         0.00163043, -0.00497734]])

In [57]:
tfidf.vocabulary_['hulk']

1562

In [59]:
from sklearn.metrics import pairwise_distances

In [62]:
dist_hulk=pairwise_distances(X=word_vec,Y=word_vec[1562,:].reshape(1,-1),metric='cosine')

In [63]:
dist_hulk.flatten().argsort()

array([1562, 3275, 1699, ..., 2472, 2213,  957])

In [64]:
id_to_word={v:k for k,v in tfidf.vocabulary_.items()}

In [66]:
id_to_word[1700]

'ironic'